In [ ]:
import datasets

# Load Geo-Bench
train_dataset = datasets.load_dataset("GEO-optim/geo-bench", 'train')
test_dataset = datasets.load_dataset("GEO-optim/geo-bench", 'test')

In [23]:
# Checking how the dataset looks

print(train_dataset['train'][0])
print(test_dataset['test'][0]['sources'][0])
print(test_dataset['test'][0])
print(test_dataset['test'][0]['query'])
print(test_dataset['test'][1]['sources'])

{'query': "how does the pokemon bank work? What can I do with it? I'm considering getting it", 'dset': 'eli5', 'sugg_idx': 3}
{'raw_text': 'Sports in KarnatakaThis style and content may require cleanup to meet Wikipedia\'s quality standards. (November 2011)Cricket is by far the most popular sport in Karnataka with International cricket matches attracting a sizeable number of spectators who are willing to pay more than the standard ticket price to get a chance to watch the match.[1] The sports related infrastructure is mainly concentrated in Bangalore which also played host to the 4th National Games of India in the year 1997.[2] Bangalore is also the location of the Sports Authority of India (SAI) which is the premier sports institute in the country.[3] Karnataka is sometimes referred to as the cradle of Indian swimming because of high standards in swimming compared to other states.[4]Association Football[edit]Amidst of cricket, which is the most popular sport of Karnataka, football fin

In [3]:
# Experimenting with Gemini's API

from google import genai
from google.genai import types
client = genai.Client()
response = client.models.generate_content(
    model="gemini-2.5-pro", 
    contents="Explain how AI works in short",
)
print(response.text)

Of course. Here is a simple explanation of how AI works.

Think of it like teaching a child to recognize a cat.

**1. You give it lots of examples (Data).**
You don't write a list of rules like "a cat has fur, four legs, and pointy ears." Instead, you just show the child thousands of pictures, pointing out, "This is a cat," "This is also a cat," and "This is a dog, *not* a cat."

**2. It learns to find patterns (Training).**
The child's brain (the AI model) starts to figure out the common features on its own. It notices that things labeled "cat" usually have whiskers, a certain eye shape, and pointy ears. It creates its own complex set of internal "rules" based on these patterns.

**3. It makes a prediction (Inference).**
Now, you show the child a picture of a new cat it has never seen before. Using the patterns it learned, it makes an educated guess: **"That's a cat!"**

---

That’s the core of how most modern AI works.

*   **ChatGPT** was trained on a massive amount of text from the

In [ ]:
# Google's latest Gemini SDK is google.genai. The following is the old one.
import google.generativeai as googlegenai
model = 'gemini-2.5-pro'
model_client = googlegenai.GenerativeModel(model)
response = model_client.generate_content(
                    "Explain how AI works in short",
                    generation_config={
                        'temperature': 0.5,
                        'top_p': 1.0,
                        'max_output_tokens': 1024,
                    }
                )
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "role": "model"
          },
          "finish_reason": "MAX_TOKENS",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 7,
        "total_token_count": 1030
      },
      "model_version": "gemini-2.5-pro"
    }),
)

In [7]:
# Pipeline which picks up queries from the test dataset and queries Gemini using the sources mentioned in the query.
import datasets
from google import genai
from google.genai import types

# Load Geo-Bench
train_dataset = datasets.load_dataset("GEO-optim/geo-bench", 'train')
test_dataset = datasets.load_dataset("GEO-optim/geo-bench", 'test')

query_prompt = """Write an accurate and concise answer for the given user question, using _only_ the provided summarized web 
search results. The answer should be correct, high-quality, and written by an expert using an unbiased and journalistic tone. 
The user's language of choice such as English, Français, Español, Deutsch, or 日本語 should be used. The answer should be 
informative, interesting, and engaging. The answer's logic and reasoning should be rigorous and defensible. Every sentence 
in the answer should be _immediately followed_ by an in-line citation to the search result(s). The cited search result(s) 
should fully support _all_ the information in the sentence. Search results need to be cited using [index]. When citing 
several search results, use [1][2][3] format rather than [1, 2, 3]. You can use multiple search results to respond 
comprehensively while avoiding irrelevant search results.

Question: {query}

Search Results:
{source_text}
"""

response = ""

for row in test_dataset['test']:
    query = row['query']
    sources = row['sources']
    print("Query: " + query)
    source_text = ""
    response = ""

    for i, s in enumerate(sources):
        source_text += "Source " + str(i + 1) + ": " + s['cleaned_text']
        source_text += "\n\n\n\n"

    print("Source Text:\n\n" + source_text)

    client = genai.Client()
    response = client.models.generate_content(
        model="gemini-2.5-pro", 
        contents=query_prompt.format(query = query, source_text = source_text),
    )

    print("Response:\n\n" + response.text)

    # Testing only one query
    break

Query: mention the names of any 3 famous folklore sports in karnataka state
Source Text:

Source 1: Sports in Karnataka
Cricket is by far the most popular sport in Karnataka with International cricket matches attracting a sizeable number of spectators who are willing to pay more than the standard ticket price to get a chance to watch the match. The sports related infrastructure is mainly concentrated in Bangalore which also played host to the 4th National Games of India in the year 1997. Bangalore is also the location of the Sports Authority of India (SAI) which is the premier sports institute in the country. Karnataka is sometimes referred to as the cradle of Indian swimming because of high standards in swimming compared to other states.

Amidst of cricket, which is the most popular sport of Karnataka, football finds its way in the state and attracts good amount of spectators during Indian Super League games of the club, Bengaluru FC. The game is also popular in the districts of Balla

ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}

In [ ]:
# Test the position-adjusted word count metric 
# Observation: it counts the punctuation marks and stylistic characters (such as '*') as words,
# leading to a miscalculation

from utils import extract_citations_new, impression_wordpos_count_simple
x = extract_citations_new(response.text)

print("Response:\n" + response.text)
print("\nExtracted citations:")
for i in x:
    print(i)

print("\nPosition-adjusted word count = " + str(impression_wordpos_count_simple(x)))

Response:
Based on the provided information, three famous folklore sports in the state of Karnataka are Kambala, Kobri Hori Sparde, and Kichchu Haisodu [2].

*   **Kambala** Kambala is an annual buffalo race and a rural folk sport traditionally organized by farming families after the harvest of the rabi crop [2][3]. The event, which takes place in paddy fields filled with slush, is held between November and March [2].
*   **Kobri Hori Sparde** Considered one of the state's most popular rural sports, Kobri Hori Sparde translates to "catch a running bull" [2]. The sport is held during Diwali and involves farmers and youngsters attempting to subdue a bull in order to tie dry coconuts around its sharpened horns [2].
*   **Kichchu Haisodu** This is a ritualistic sport associated with farming that is observed during Makara Sankranti in the old Mysuru region, including areas around Bengaluru, Mandya, and Hassan [2]. In this event, cattle are decorated, their horns are painted, and they are ma

In [6]:
# Now added the helper function to disregard the 

# Previous response:
# Extracted citations:
# [(['Based', 'on', 'the', 'provided', 'information', ',', 'three', 'famous', 'folklore', 'sports', 'in', 'the', 'state', 'of', 'Karnataka', 'are', 'Kambala', ',', 'Kobri', 'Hori', 'Sparde', ',', 'and', 'Kichchu', 'Haisodu', '[', '2', ']', '.'], 'Based on the provided information, three famous folklore sports in the state of Karnataka are Kambala, Kobri Hori Sparde, and Kichchu Haisodu [2].', [2])]
# [(['*', '*', '*', 'Kambala', '*', '*', 'Kambala', 'is', 'an', 'annual', 'buffalo', 'race', 'and', 'a', 'rural', 'folk', 'sport', 'traditionally', 'organized', 'by', 'farming', 'families', 'after', 'the', 'harvest', 'of', 'the', 'rabi', 'crop', '[', '2', ']', '[', '3', ']', '.'], '*   **Kambala** Kambala is an annual buffalo race and a rural folk sport traditionally organized by farming families after the harvest of the rabi crop [2][3].', [2, 3]), (['The', 'event', ',', 'which', 'takes', 'place', 'in', 'paddy', 'fields', 'filled', 'with', 'slush', ',', 'is', 'held', 'between', 'November', 'and', 'March', '[', '2', ']', '.'], 'The event, which takes place in paddy fields filled with slush, is held between November and March [2].', [2]), (['*', '*', '*', 'Kobri', 'Hori', 'Sparde', '*', '*', 'Considered', 'one', 'of', 'the', 'state', "'s", 'most', 'popular', 'rural', 'sports', ',', 'Kobri', 'Hori', 'Sparde', 'translates', 'to', '``', 'catch', 'a', 'running', 'bull', "''", '[', '2', ']', '.'], '*   **Kobri Hori Sparde** Considered one of the state\'s most popular rural sports, Kobri Hori Sparde translates to "catch a running bull" [2].', [2]), (['The', 'sport', 'is', 'held', 'during', 'Diwali', 'and', 'involves', 'farmers', 'and', 'youngsters', 'attempting', 'to', 'subdue', 'a', 'bull', 'in', 'order', 'to', 'tie', 'dry', 'coconuts', 'around', 'its', 'sharpened', 'horns', '[', '2', ']', '.'], 'The sport is held during Diwali and involves farmers and youngsters attempting to subdue a bull in order to tie dry coconuts around its sharpened horns [2].', [2]), (['*', '*', '*', 'Kichchu', 'Haisodu', '*', '*', 'This', 'is', 'a', 'ritualistic', 'sport', 'associated', 'with', 'farming', 'that', 'is', 'observed', 'during', 'Makara', 'Sankranti', 'in', 'the', 'old', 'Mysuru', 'region', ',', 'including', 'areas', 'around', 'Bengaluru', ',', 'Mandya', ',', 'and', 'Hassan', '[', '2', ']', '.'], '*   **Kichchu Haisodu** This is a ritualistic sport associated with farming that is observed during Makara Sankranti in the old Mysuru region, including areas around Bengaluru, Mandya, and Hassan [2].', [2]), (['In', 'this', 'event', ',', 'cattle', 'are', 'decorated', ',', 'their', 'horns', 'are', 'painted', ',', 'and', 'they', 'are', 'made', 'to', 'jump', 'over', 'a', 'fire', ',', 'a', 'practice', 'farmers', 'believe', 'kills', 'insects', 'on', 'the', 'animals', "'", 'bodies', '[', '2', ']', '.'], "In this event, cattle are decorated, their horns are painted, and they are made to jump over a fire, a practice farmers believe kills insects on the animals' bodies [2].", [2])]
# 
# Position-adjusted word count = [0.0, 0.9073443266149541, 0.09265567338504593, 0.0, 0.0]

from utils import extract_citations_new, impression_wordpos_count_simple
response = """Based on the provided information, three famous folklore sports in the state of Karnataka are Kambala, Kobri Hori Sparde, and Kichchu Haisodu [2].

*   **Kambala** Kambala is an annual buffalo race and a rural folk sport traditionally organized by farming families after the harvest of the rabi crop [2][3]. The event, which takes place in paddy fields filled with slush, is held between November and March [2].
*   **Kobri Hori Sparde** Considered one of the state's most popular rural sports, Kobri Hori Sparde translates to "catch a running bull" [2]. The sport is held during Diwali and involves farmers and youngsters attempting to subdue a bull in order to tie dry coconuts around its sharpened horns [2].
*   **Kichchu Haisodu** This is a ritualistic sport associated with farming that is observed during Makara Sankranti in the old Mysuru region, including areas around Bengaluru, Mandya, and Hassan [2]. In this event, cattle are decorated, their horns are painted, and they are made to jump over a fire, a practice farmers believe kills insects on the animals' bodies [2]."""

x = extract_citations_new(response)

print("Response:\n" + response)
print("\nExtracted citations:")
for i in x:
    print(i)

print("\nPosition-adjusted word count = " + str(impression_wordpos_count_simple(x)))

Response:
Based on the provided information, three famous folklore sports in the state of Karnataka are Kambala, Kobri Hori Sparde, and Kichchu Haisodu [2].

*   **Kambala** Kambala is an annual buffalo race and a rural folk sport traditionally organized by farming families after the harvest of the rabi crop [2][3]. The event, which takes place in paddy fields filled with slush, is held between November and March [2].
*   **Kobri Hori Sparde** Considered one of the state's most popular rural sports, Kobri Hori Sparde translates to "catch a running bull" [2]. The sport is held during Diwali and involves farmers and youngsters attempting to subdue a bull in order to tie dry coconuts around its sharpened horns [2].
*   **Kichchu Haisodu** This is a ritualistic sport associated with farming that is observed during Makara Sankranti in the old Mysuru region, including areas around Bengaluru, Mandya, and Hassan [2]. In this event, cattle are decorated, their horns are painted, and they are ma

In [38]:
from utils import extract_citations_new, impression_subjective_impression, impression_wordpos_count_simple, impression_subjpos_detailed, impression_diversity_detailed, impression_uniqueness_detailed, impression_follow_detailed, impression_influence_detailed, impression_relevance_detailed, impression_subjcount_detailed, impression_pos_count_simple, impression_word_count_simple

IMPRESSION_FNS = {
	'simple_wordpos' : impression_wordpos_count_simple, 
	'simple_word' : impression_word_count_simple,
	'simple_pos' : impression_pos_count_simple,
	'subjective_score' : impression_subjective_impression,
	'subjpos_detailed' : impression_subjpos_detailed,
	'diversity_detailed' : impression_diversity_detailed,
	'uniqueness_detailed' : impression_uniqueness_detailed,
	'follow_detailed' : impression_follow_detailed,
	'influence_detailed' : impression_influence_detailed,
	'relevance_detailed' : impression_relevance_detailed,
	'subjcount_detailed' : impression_subjcount_detailed,
}

scores = [
            IMPRESSION_FNS['simple_wordpos'](extract_citations_new(response.text)), 
            IMPRESSION_FNS['subjpos_detailed'](response.text, query), 
            IMPRESSION_FNS['follow_detailed'](response.text, query), 
            IMPRESSION_FNS['influence_detailed'](response.text, query), 
            IMPRESSION_FNS['subjcount_detailed'](response.text, query)
        ]

print(scores)

[[0.0, 0.9073443266149541, 0.09265567338504593, 0.0, 0.0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
